#### langchain 예제

In [ ]:
# 사전설치 : pip install faiss-cpu, sentence_transformers, community
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import HuggingFaceEmbeddings

In [4]:
vectorstore = FAISS.from_texts(
    [
        "영준은 랭체인 주식회사에서 근무를 하였습니다.",
        "설현은 테디와 같은 회사에서 근무하였습니다.",
        "영준의 직업은 개발자입니다.",
        "설현의 직업은 디자이너입니다.",
    ],
    embedding = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask'),
)

C:\Users\yj\AppData\Local\Temp\ipykernel_10208\3793202338.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name='jhgan/ko-sroberta-multitask'),
c:\Python310\aiproject\llm_Study\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Python310\aiproject\llm_Study\.venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yj\.cache\huggingface\hub\models--jhgan--ko-sroberta-multitask. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [16]:
retrieval = vectorstore.as_retriever()

In [17]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [18]:
prompt = ChatPromptTemplate.from_template(template)

In [19]:
from langchain_community.llms import Ollama

In [20]:
# model = Ollama(model = "llama3:8b")
model = Ollama(model = "gemma2")

In [21]:
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [22]:
retrieval_chain.invoke("설현의 직업은?")

'설현의 직업은 디자이너입니다.  \n'

In [23]:
retrieval_chain.invoke("영준이 근무한 곳은?")

'영준은 랭체인 주식회사에서 근무를 하였습니다.  \n'

In [25]:
retrieval_chain.invoke("영준이 개발자라면 유망한 SW 기술 추천해줘.")

"The provided context describes Youngjun as a developer and mentions his past employment at a blockchain company.  Based on this, here are some promising SW technologies I recommend:\n\n* **Blockchain Development:** Given Youngjun's experience at a blockchain company, specializing in blockchain development would be highly relevant and in demand. \n* **Smart Contract Development:** This is a crucial aspect of blockchain technology and offers good career prospects.\n* **Web3 Development:**  This emerging field focuses on decentralized applications (dApps) and technologies built on blockchains.\n\nRemember, the best technology to focus on depends on individual interests and market trends. \n\n\n"